# Preambule

In [1]:
library(stringr)
library(tidyverse)
library(nlme)

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats

Attaching package: ‘nlme’

The following object is masked from ‘package:dplyr’:

    collapse



In [16]:
sessionInfo()

R version 3.4.1 (2017-06-30)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 18.04.3 LTS

Matrix products: default
BLAS: /ebio/abt3_projects/small_projects/aruaud/miniconda3/envs/test/lib/R/lib/libRblas.so
LAPACK: /ebio/abt3_projects/small_projects/aruaud/miniconda3/envs/test/lib/R/lib/libRlapack.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] nlme_3.1-131    dplyr_0.7.4     purrr_0.2.3     readr_1.1.1    
[5] tidyr_0.7.1     tibble_1.3.4    ggplot2_2.2.1   tidyverse_1.1.1
[9] stringr_1.2.0  

loaded via a namespace (and not attached)

# SCFA: two-way anova 

In [16]:
scfa <- read.table('../data/SCFA_wt.txt') %>% subset(str_detect(Sample, pattern = 'C') & Time > 6*24 ) %>% unique

In [17]:
scfa$condition <- ifelse(str_detect(scfa$SampleBatch, pattern = 'C3'), 'N2-2bar'
                        , ifelse(str_detect(scfa$SampleBatch, pattern = 'Catm'), 'N2-atm'
                                 , 'H2-2bar'
                        ) )


## Acetate

In [7]:
mod_ace <- lm(Concentration ~ condition * Sample, data = scfa %>% subset(Name == 'Acetate'))
ano_ace <- aov(mod_ace)
ano_ace %>% summary

                 Df Sum Sq Mean Sq F value   Pr(>F)    
condition         2  4.038   2.019   73.00 1.92e-07 ***
Sample            1  8.778   8.778  317.41 5.36e-10 ***
condition:Sample  2  1.609   0.805   29.09 2.50e-05 ***
Residuals        12  0.332   0.028                     
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [9]:
TukeyHSD(ano_ace)

  Tukey multiple comparisons of means
    95% family-wise confidence level

Fit: aov(formula = mod_ace)

$condition
                  diff        lwr        upr     p adj
N2-2bar-H2-2bar  1.127  0.8708501  1.3831499 0.0000002
N2-atm-H2-2bar   0.325  0.0688501  0.5811499 0.0138898
N2-atm-N2-2bar  -0.802 -1.0581499 -0.5458501 0.0000067

$Sample
         diff     lwr      upr p adj
MC-C 1.396667 1.22586 1.567473     0

$`condition:Sample`
                            diff        lwr         upr     p adj
N2-2bar:C-H2-2bar:C    0.7886667  0.3325821  1.24475119 0.0009104
N2-atm:C-H2-2bar:C    -0.4066667 -0.8627512  0.04941786 0.0912975
H2-2bar:MC-H2-2bar:C   0.6833333  0.2272488  1.13941786 0.0030698
N2-2bar:MC-H2-2bar:C   2.1486667  1.6925821  2.60475119 0.0000000
N2-atm:MC-H2-2bar:C    1.7400000  1.2839155  2.19608452 0.0000003
N2-atm:C-N2-2bar:C    -1.1953333 -1.6514179 -0.73924881 0.0000162
H2-2bar:MC-N2-2bar:C  -0.1053333 -0.5614179  0.35075119 0.9666277
N2-2bar:MC-N2-2bar:C   1.3600000

In [34]:
### To calculate the difference between mono- and co-cultures

tmp1 <- scfa %>% subset(Name == 'Acetate' & !str_detect(Replicate, pattern = 'M')) %>% 
            group_by(SampleBatch) %>% summarize(avg = mean(Concentration))
tmp1$Batch <- tmp1$SampleBatch %>% str_replace(pattern = '^C', replacement = '')

tmp2 <- scfa %>% subset(Name == 'Acetate' & str_detect(Replicate, pattern = 'M') ) 
tmp2$Batch <- tmp2$SampleBatch %>% str_replace(pattern = '^MC', replacement = '')
tmp2 <- tmp2 %>% left_join(tmp1, by = 'Batch')
tmp2$Con_centred <- tmp2$Concentration - tmp2$avg

In [44]:
tmp2$Con_centred %>% mean

[1] 1.396667

## Butyrate

In [10]:
mod_but <- lm(Concentration ~ condition * Sample, data = scfa %>% subset(Name == 'Butyrate'))
ano_but <- aov(mod_but)
ano_but %>% summary

                 Df Sum Sq Mean Sq F value   Pr(>F)    
condition         2  5.658   2.829  89.402 6.19e-08 ***
Sample            1  5.109   5.109 161.461 2.55e-08 ***
condition:Sample  2  0.055   0.027   0.862    0.447    
Residuals        12  0.380   0.032                     
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [11]:
TukeyHSD(ano_but)

  Tukey multiple comparisons of means
    95% family-wise confidence level

Fit: aov(formula = mod_but)

$condition
                     diff         lwr        upr     p adj
N2-2bar-H2-2bar  0.200000 -0.07400074  0.4740007 0.1680536
N2-atm-H2-2bar  -1.076667 -1.35066741 -0.8026659 0.0000006
N2-atm-N2-2bar  -1.276667 -1.55066741 -1.0026659 0.0000001

$Sample
          diff       lwr        upr p adj
MC-C -1.065556 -1.248266 -0.8828455     0

$`condition:Sample`
                            diff        lwr        upr     p adj
N2-2bar:C-H2-2bar:C    0.1000000 -0.3878686  0.5878686 0.9798798
N2-atm:C-H2-2bar:C    -1.0483333 -1.5362019 -0.5604647 0.0001203
H2-2bar:MC-H2-2bar:C  -1.1133333 -1.6012019 -0.6254647 0.0000664
N2-2bar:MC-H2-2bar:C  -0.8133333 -1.3012019 -0.3254647 0.0012536
N2-atm:MC-H2-2bar:C   -2.2183333 -2.7062019 -1.7304647 0.0000000
N2-atm:C-N2-2bar:C    -1.1483333 -1.6362019 -0.6604647 0.0000487
H2-2bar:MC-N2-2bar:C  -1.2133333 -1.7012019 -0.7254647 0.0000278
N2-2bar:MC-N2-

In [37]:
mean(c(-1.1, -0.9, -1.1))

[1] -1.033333

## adjusted p-value of the anovas

In [40]:
pval <- c(1.92e-07 #ace condition
          , 5.36e-10 #ace sample
          , 2.50e-05 #ace int
          
          , 6.19e-08 #but condition
          , 2.55e-08 #but sample
          , 0.447#but int
         )

In [41]:
p.adjust(pval, method = 'BH')

[1] 2.880e-07 3.216e-09 3.000e-05 1.238e-07 7.650e-08 4.470e-01

# Gases

In [2]:
ggc_all <- read.table('Gas_wt.txt', stringsAsFactors = FALSE)

In [3]:
ggc <- ggc_all %>% subset((Time == 157 & microbe %in% c('C. minuta', 'B. thetaiotaomicron') & Batch == 'N2 - 2 bar' & gas == 'H2')
                            | (Time > 144 & Time < 168 & microbe == 'M. smithii' & Batch == 'N2 - atm' & gas == 'CH4')
                            | (Time == 157 & microbe == 'M. smithii' & Batch == 'N2 - 2 bar' & gas == 'CH4')
                            | (Time > 144 & Time < 168 & microbe == 'C. minuta'& Batch == 'N2 - atm'& gas == 'H2')
                            
                            | (Time > 144 & Time < 168 & str_detect(SampleBatch, pattern = 'M(C|B)')& gas == 'CH4')
                            
              )

In [4]:
#ggc$condition[ggc$SampleBatch == 'MC-op' ] <- 'CH2-2bar'
ggc$condition <- ggc$condition %>% str_replace_all(pattern = '-', replacement = '_')

## N2 - 2 bar: C, B, MC and MB

In [5]:
### H2 production C. minuta vs B. thetaiotamicron
tmp <- c('B. thetaiotaomicron', 'C. minuta')
wilcox.test(Concentration ~ SampleBatch
            , data = ggc %>% subset(gas == 'H2' & Batch == 'N2 - 2 bar' & microbe %in% tmp) )


	Wilcoxon rank sum test

data:  Concentration by SampleBatch
W = 0, p-value = 0.1
alternative hypothesis: true location shift is not equal to 0


In [11]:
### CH4 production of M. smithii with C. minuta vs witha B. thetaiotamicron
# Comment: the names of the co-cultures both have a '/' so they are selected this way in the subset()
tmp <- c('M. smithii / B. thetaiotaomicron', 'M. smithii / C. minuta')
wilcox.test(Concentration ~ SampleBatch
            , data = ggc %>% subset(gas == 'CH4' & Batch == 'N2 - 2 bar' & str_detect(microbe, pattern = '/') ) )


	Wilcoxon rank sum test

data:  Concentration by SampleBatch
W = 0, p-value = 0.1
alternative hypothesis: true location shift is not equal to 0


## MC, C and M

In [12]:
ggc$Sample <- ggc$SampleBatch %>% str_extract(pattern = '.*(?=\\-|3)')

In [13]:
ggc$groupCH4 <- str_c(ggc$Sample, ggc$condition)

In [14]:
mod_ch4 <- lm(Concentration ~ groupCH4, data = ggc %>% subset(gas == 'CH4' & !str_detect(SampleBatch, pattern = 'B')))
ano_ch4 <- aov(mod_ch4)
ano_ch4 %>% summary

            Df Sum Sq Mean Sq F value  Pr(>F)   
groupCH4     4  159.3   39.83   6.606 0.00722 **
Residuals   10   60.3    6.03                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [15]:
TukeyHSD(ano_ch4)

  Tukey multiple comparisons of means
    95% family-wise confidence level

Fit: aov(formula = mod_ch4)

$groupCH4
                           diff        lwr       upr     p adj
MCN2_2bar-MCH2_2bar  -8.4040888 -15.002631 -1.805546 0.0124845
MCN2_atm-MCH2_2bar   -7.6395550 -14.238097 -1.041013 0.0223603
MH2H2_2bar-MCH2_2bar -5.1650888 -11.763631  1.433453 0.1488337
MH2H2_atm-MCH2_2bar  -8.8187092 -15.417252 -2.220167 0.0091472
MCN2_atm-MCN2_2bar    0.7645338  -5.834009  7.363076 0.9947654
MH2H2_2bar-MCN2_2bar  3.2390000  -3.359542  9.837542 0.5203963
MH2H2_atm-MCN2_2bar  -0.4146204  -7.013163  6.183922 0.9995164
MH2H2_2bar-MCN2_atm   2.4744662  -4.124076  9.073008 0.7334141
MH2H2_atm-MCN2_atm   -1.1791542  -7.777697  5.419388 0.9738486
MH2H2_atm-MH2H2_2bar -3.6536204 -10.252163  2.944922 0.4128934
